# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# The following block of code is an attempt to create a keyspace named 'udacity' if it doesn't already exist.
# Keyspaces in Apache Cassandra are analogous to databases in traditional relational databases.

# The 'try' block is used to catch any exceptions that may occur during the execution of the code.
try:
    # Using the Cassandra session.execute() method to execute a CQL (Cassandra Query Language) command.
    # The command is creating a keyspace named 'udacity' if it doesn't already exist.
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )

# If an exception occurs during the execution of the code within the 'try' block, it will be caught here.
except Exception as e:
    # Print the exception that occurred. This helps in identifying and debugging issues.
    print(e)


#### Set Keyspace

In [7]:
# The following block of code is an attempt to set the keyspace for the Cassandra session to 'udacity'.
# This is crucial because subsequent queries and operations will be performed within this keyspace.

# The 'try' block is used to catch any exceptions that may occur during the execution of the code.
try:
    # Using the Cassandra session.set_keyspace() method to set the keyspace to 'udacity'.
    session.set_keyspace('udacity')

# If an exception occurs during the execution of the code within the 'try' block, it will be caught here.
except Exception as e:
    # Print the exception that occurred. This helps in identifying and debugging issues.
    print(e)



##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Create a table named 'music_app_history' with sessionId as the partition key
# and itemInSession as the clustering column
query = "CREATE TABLE IF NOT EXISTS music_app_history"
query += "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"
session.execute(query)


In [9]:
# The following block of code reads data from a CSV file ('event_datafile_new.csv')
# and inserts the relevant information into the 'music_app_history' table in Apache Cassandra.

# Specify the path to the CSV file
file = 'event_datafile_new.csv'

# Open the CSV file in read mode with UTF-8 encoding
with open(file, encoding='utf8') as f:
    # Create a CSV reader object
    csvreader = csv.reader(f)
    
    # Skip the header row in the CSV file
    next(csvreader)
    
    # Iterate through each line in the CSV file
    for line in csvreader:
        # Construct the INSERT statement for the 'music_app_history' table
        query = "INSERT INTO music_app_history(sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        # Execute the INSERT statement with data from the current line of the CSV file
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


In [10]:
# The following block of code is a SELECT statement to verify that data was successfully entered
# into the 'music_app_history' table in Apache Cassandra.

# Construct the SELECT statement to retrieve data for a specific sessionId and itemInSession
query = "SELECT artist, song, length FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4"

# Execute the SELECT statement and retrieve the result set
rows = session.execute(query)

# Iterate through the result set and print the artist, song, and length
for row in rows:
    # Access the columns of the result row using dot notation
    print(row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
# The following block of code is creating a table named 'artist_song_user_music_app_history'
# in Apache Cassandra. This table is designed to store information about the artist, song, and user
# for a given userId and sessionId, sorted by itemInSession.

# The CQL (Cassandra Query Language) command for creating the table is constructed as a string.
query = "CREATE TABLE IF NOT EXISTS artist_song_user_music_app_history "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY (userId, sessionId), itemInSession )"

# Using the Cassandra session.execute() method to execute the CQL command and create the table.
session.execute(query)

# Specify the path to the CSV file
file = 'event_datafile_new.csv'

# Open the CSV file in read mode with UTF-8 encoding
with open(file, encoding='utf8') as f:
    # Create a CSV reader object
    csvreader = csv.reader(f)
    
    # Skip the header row in the CSV file
    next(csvreader)
    
    # Iterate through each line in the CSV file
    for line in csvreader:
        # Construct the INSERT statement for the 'artist_song_user_music_app_history' table
        query = "INSERT INTO artist_song_user_music_app_history (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Execute the INSERT statement with data from the current line of the CSV file
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))


In [12]:
# The following block of code is a SELECT statement to retrieve every user name
# (first and last) in the 'artist_song_user_music_app_history' table
# for users who listened to the song 'All Hands Against His Own'.

# Construct the SELECT statement to retrieve user names for a specific song
query = "SELECT artist, song, firstName, lastName FROM artist_song_user_music_app_history WHERE song = 'All Hands Against His Own'"

# Execute the SELECT statement and retrieve the result set
rows = session.execute(query)

# Iterate through the result set and print the artist, song, firstName, and lastName
for row in rows:
    # Access the columns of the result row using dot notation
    print(row.artist, row.song, row.firstname, row.lastname)


Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
# The following block of code is creating a table named 'user_song_song_app_history'
# in Apache Cassandra. This table is designed to store information about the song, userId,
# and user names (first and last) for a given song.

# The CQL (Cassandra Query Language) command for creating the table is constructed as a string.
query = "CREATE TABLE IF NOT EXISTS user_song_song_app_history "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"

# Using the Cassandra session.execute() method to execute the CQL command and create the table.
session.execute(query)

# Specify the path to the CSV file
file = 'event_datafile_new.csv'

# Open the CSV file in read mode with UTF-8 encoding
with open(file, encoding='utf8') as f:
    # Create a CSV reader object
    csvreader = csv.reader(f)
    
    # Skip the header row in the CSV file
    next(csvreader)
    
    # Iterate through each line in the CSV file
    for line in csvreader:
        # Construct the INSERT statement for the 'user_song_song_app_history' table
        query = "INSERT INTO user_song_song_app_history (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        # Execute the INSERT statement with data from the current line of the CSV file
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


In [14]:
# The following block of code is a SELECT statement to retrieve every user name
# (first and last) in the 'user_song_song_app_history' table
# for users who listened to the song 'All Hands Against His Own'.

# Construct the SELECT statement to retrieve user names for a specific song
query = "SELECT firstName, lastName FROM user_song_song_app_history WHERE song='All Hands Against His Own'"

# Execute the SELECT statement and retrieve the result set
rows = session.execute(query)

# Iterate through the result set and print the firstName and lastName
for row in rows:
    # Access the columns of the result row using dot notation
    print(row.firstname, row.lastname)


Jacqueline Lynch
Tegan Levine
Sara Johnson


In [17]:
# The following block of code is using the Cassandra session.execute() method
# to drop the tables 'music_app_history', 'artist_song_user_music_app_history', and 'user_song_song_app_history'.

# Drop the 'music_app_history' table
session.execute("DROP TABLE IF EXISTS music_app_history")

# Drop the 'artist_song_user_music_app_history' table
session.execute("DROP TABLE IF EXISTS artist_song_user_music_app_history")

# Drop the 'user_song_song_app_history' table
session.execute("DROP TABLE IF EXISTS user_song_song_app_history")


In [18]:
# The following block of code is used to shut down the Cassandra session and cluster connections.

# Shut down the Cassandra session
session.shutdown()

# Shut down the Cassandra cluster
cluster.shutdown()
